In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_pickle('data/cleaned_data.pkl')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
X= df.drop(["Class"], axis=1)
y= df["Class"]

In [5]:
## Splitting the Train and test

y = y.values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
len(y_train[y_train == 1])

394

## Logistic Regression

In [8]:
model_lr = LogisticRegressionCV()

In [9]:
model_lr.fit(X_train, y_train)

/Users/karankaran/Desktop/Github_Karan/CreditCard_FraudDetection/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/karankaran/Desktop/Github_Karan/CreditCard_FraudDetection/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/karankaran/Desktop/Github_Karan/CreditCard_FraudDetection/venv/lib/python3.11/site-packages/sklearn/l

LogisticRegressionCV()

In [10]:
y_pred = model_lr.predict(X_test)

In [11]:
print('Accuracy score:', accuracy_score(y_pred,y_test)*100)
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))

Accuracy score: 99.90519995786666
[[56853    43]
 [   11    55]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56896
           1       0.56      0.83      0.67        66

    accuracy                           1.00     56962
   macro avg       0.78      0.92      0.84     56962
weighted avg       1.00      1.00      1.00     56962



In [12]:
type(y_test)

numpy.ndarray

In [13]:
len(y_test[y_test == 1])

98

In [14]:
len(y_pred[y_pred ==1])

66

In [15]:
print(y_pred.shape)
print(y_test.shape)


(56962,)
(56962, 1)


In [16]:
y_pred = y_pred.reshape(-1, 1)

In [28]:
df_out = pd.DataFrame({"y_pred": y_pred, "y_test": y_test})

ValueError: Per-column arrays must each be 1-dimensional

In [24]:
df_out

array([[[1],
        [1]],

       [[0],
        [0]],

       [[0],
        [0]],

       ...,

       [[0],
        [0]],

       [[0],
        [0]],

       [[0],
        [0]]])